This code doesn't create a cluster and run jobs on it. It just manipulates a PySpark Dataframe.

In [1]:
print("Hi Gany, This is a sample PySpark installation")

Hi Gany, This is a sample PySpark installation


In [2]:
#This doesn't work from the notebook, but works from the command line
!brew install apache-spark
#This doesn't help either from here or the command line. Download java through the browser.
# But looks like the brew install from command line actually worked.. look at the PATH and 
# JAVA_HOME variables in the below cell. They are brew based.
!brew install openjdk@11

==> Downloading https://formulae.brew.sh/api/formula.jws.json

==> Downloading https://formulae.brew.sh/api/cask.jws.json

To reinstall 3.5.0, run:
  brew reinstall apache-spark
To reinstall 11.0.21, run:
  brew reinstall openjdk@11


In [3]:
!export SPARK_HOME=/opt/homebrew/Cellar/apache-spark/3.3.0/libexec
!export PATH=/opt/homebrew/Cellar/apache-spark/3.3.0/bin:$PATH
!export PATH=/opt/homebrew/opt/openjdk@11/bin/java:$PATH
# looks the case of the JAVA_HOME variable made all the difference    
!export JAVA_HOME=/opt/homebrew/opt/openjdk@11/bin/java
!export PYSPARK_SUBMIT_ARGS="--master local[3] pyspark-shell"

In [4]:
!java -version

openjdk version "11.0.21" 2023-10-17
OpenJDK Runtime Environment Homebrew (build 11.0.21+0)
OpenJDK 64-Bit Server VM Homebrew (build 11.0.21+0, mixed mode)


In [5]:
!pip3 install pyspark
!pip3 install findspark
!pip3 install pyarrow



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [6]:
import findspark
findspark.init()

In [7]:
# To create a mock dataframe
columns = ["language","users_count"]
data = [("Java", "20000"), ("Python", "100000"), ("Scala", "3000")]

In [8]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

df = spark.createDataFrame(data,columns)
df.show()
type(df)

24/01/06 21:37:50 WARN Utils: Your hostname, Ganapathys-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
24/01/06 21:37:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/06 21:37:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+



pyspark.sql.dataframe.DataFrame

#  The below 2 cells are an illustration of PySpark Pandas dataframes

So there are these 3 types of dataframes:
	1. Pandas Dataframe
	2. PySpark Dataframe- https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html
	3. Pandas on Spark Dataframe -https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_ps.html

In [9]:
import pyspark.pandas as ps
import pyarrow

psdf = ps.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

psdf


/opt/homebrew/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


In [10]:
type(psdf)

pyspark.pandas.frame.DataFrame

# Back to Regular PySpark DataFrames

In [11]:
challenge_df = spark.read.format("csv").option("header", "true").load("/Users/ganapathychidambaram/challenge.csv")

In [12]:
challenge_df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [13]:
# This import is needed to run the when method etc. 
from pyspark.sql.functions import *
challenge_df = challenge_df.withColumn("Mexico", when(challenge_df.Country == "Mexico", "Yes").otherwise("No"))
challenge_df.show()

+---------------+--------------+-----------------+----------+------+
|     ip_address|       Country|      Domain Name|Bytes_used|Mexico|
+---------------+--------------+-----------------+----------+------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|    No|
| 119.239.207.13|         China|         youtu.be|        51|    No|
|  68.69.217.210|         China|        adobe.com|        10|    No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|    No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|    No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|    No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|    No|
| 127.242.24.138|         China| surveymonkey.com|       123|    No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|    No|
|   237.54.11.63|         China|       amazon.com|        83|    No|
| 252.141.157.25|         Japan|      cornell.edu|       374|    No|
|185.220.128.248|       Belgium|  

In [14]:
bytes_agg_df = challenge_df.groupBy("Mexico").agg(sum("Bytes_used"))
bytes_agg_df.show()

+------+---------------+
|Mexico|sum(Bytes_used)|
+------+---------------+
|    No|       508076.0|
|   Yes|         6293.0|
+------+---------------+



In [15]:
unique_ip_count_df = challenge_df.groupBy("Country").agg(countDistinct("ip_address"))
unique_ip_count_df.show()

+-----------+-----------------+
|    Country|count(ip_address)|
+-----------+-----------------+
|       Chad|                1|
|     Russia|               56|
|   Paraguay|                1|
|      Yemen|                1|
|     Sweden|               28|
|Philippines|               65|
|   Malaysia|                5|
|     Turkey|                1|
|     Malawi|                2|
|    Germany|                5|
|    Comoros|                1|
|Afghanistan|                5|
|     Rwanda|                1|
|      Sudan|                1|
|     France|               21|
|     Greece|                8|
|  Sri Lanka|                3|
|   Dominica|                1|
|  Argentina|               14|
|    Belgium|                1|
+-----------+-----------------+
only showing top 20 rows



# The below part deals with RDDs; everything above was PySpark Dataframes

In [16]:
type(spark)

pyspark.sql.session.SparkSession

In [17]:
#Retrieving the context from the session; you cannot have 2 spark contexts in the same program
sc = spark.sparkContext

In [18]:
%%writefile example.txt
first line
second line
third line
fourth line

Writing example.txt


In [19]:
text_file_rdd = sc.textFile('example.txt')

In [20]:
text_file_rdd.first()

'first line'

In [21]:
# The new RDD is not yet created; lazy evaluation
find_seconds_rdd = text_file_rdd.filter(lambda line: "second" in line)

In [22]:
# It is created now
find_seconds_rdd.collect()

['second line']

In [23]:
csv_file_rdd = sc.textFile("/Users/ganapathychidambaram/challenge.csv")

In [24]:
csv_file_rdd.collect()

['ip_address,Country,Domain Name,Bytes_used',
 '52.81.192.172,China,odnoklassniki.ru,463',
 '119.239.207.13,China,youtu.be,51',
 '68.69.217.210,China,adobe.com,10',
 '7.191.21.223,Bulgaria,linkedin.com,853',
 '211.13.10.68,Indonesia,hud.gov,29',
 '239.80.21.97,Suriname,smh.com.au,218',
 '106.214.106.233,Jamaica,amazonaws.com,95',
 '127.242.24.138,China,surveymonkey.com,123',
 '99.2.6.139,Czech Republic,geocities.jp,322',
 '237.54.11.63,China,amazon.com,83',
 '252.141.157.25,Japan,cornell.edu,374',
 '185.220.128.248,Belgium,weebly.com,389',
 '151.77.19.45,Afghanistan,independent.co.uk,282',
 '9.161.158.225,Indonesia,bloglines.com,726',
 '156.144.61.155,Czech Republic,slideshare.net,657',
 '8.96.188.151,Indonesia,ibm.com,517',
 '5.72.7.65,Mexico,youtu.be,877',
 '227.110.112.144,Croatia,ehow.com,287',
 '81.71.28.97,Thailand,last.fm,588',
 '9.255.129.184,Thailand,mtv.com,114',
 '209.166.62.189,Indonesia,cloudflare.com,595',
 '35.116.221.230,Russia,skype.com,960',
 '221.202.66.93,Philippine

In [25]:
# Split the lines into words
words_rdd = csv_file_rdd.map(lambda x: x.split(','))
# Select the country and Bytes_used column - exactly 2 columns needed for reduceByKey
pairs = words_rdd.map(lambda x: [x[1], x[3]])
pairs.collect()

[['Country', 'Bytes_used'],
 ['China', '463'],
 ['China', '51'],
 ['China', '10'],
 ['Bulgaria', '853'],
 ['Indonesia', '29'],
 ['Suriname', '218'],
 ['Jamaica', '95'],
 ['China', '123'],
 ['Czech Republic', '322'],
 ['China', '83'],
 ['Japan', '374'],
 ['Belgium', '389'],
 ['Afghanistan', '282'],
 ['Indonesia', '726'],
 ['Czech Republic', '657'],
 ['Indonesia', '517'],
 ['Mexico', '877'],
 ['Croatia', '287'],
 ['Thailand', '588'],
 ['Thailand', '114'],
 ['Indonesia', '595'],
 ['Russia', '960'],
 ['Philippines', '570'],
 ['Philippines', '16'],
 ['Palestinian Territory', '222'],
 ['Russia', '492'],
 ['Georgia', '179'],
 ['Brazil', '681'],
 ['Portugal', '649'],
 ['China', '511'],
 ['Indonesia', '276'],
 ['Brazil', '342'],
 ['Canada', '190'],
 ['Russia', '79'],
 ['Australia', '563'],
 ['France', '968'],
 ['China', '333'],
 ['Laos', '707'],
 ['Indonesia', '629'],
 ['United States', '967'],
 ['Luxembourg', '714'],
 ['Poland', '951'],
 ['Indonesia', '728'],
 ['Indonesia', '496'],
 ['China', 

In [26]:
# Equivalent of group by in SQL, 
bytes_by_country = pairs.reduceByKey(lambda x,y: int(x) + int(y)).collect()
bytes_by_country

[('Country', 'Bytes_used'),
 ('China', 83398),
 ('Bulgaria', 2447),
 ('Czech Republic', 12758),
 ('Japan', 12432),
 ('Afghanistan', 2538),
 ('Mexico', 6293),
 ('Russia', 32193),
 ('Philippines', 30338),
 ('Australia', 1774),
 ('France', 10559),
 ('Laos', 799),
 ('Germany', 2575),
 ('Sweden', 15422),
 ('Nigeria', 6097),
 ('El Salvador', '378'),
 ('Benin', '612'),
 ('Libya', 48),
 ('Chile', 3388),
 ('Bangladesh', 1776),
 ('Bahrain', 877),
 ('Saudi Arabia', 353),
 ('Macedonia', 2755),
 ('Dominican Republic', 842),
 ('Uzbekistan', '181'),
 ('Cyprus', 749),
 ('Ukraine', 8483),
 ('Botswana', '401'),
 ('Egypt', 884),
 ('Tanzania', 1027),
 ('Morocco', 3311),
 ('Israel', 2300),
 ('Uganda', 1915),
 ('Nepal', '231'),
 ('Hungary', 1376),
 ('Guam', 1250),
 ('Ecuador', 1711),
 ('Syria', 2361),
 ('Lebanon', 1315),
 ('Burkina Faso', 2362),
 ('Serbia', 3218),
 ('Ethiopia', 1190),
 ('Sri Lanka', 1918),
 ('Honduras', 1552),
 ('Nicaragua', 3496),
 ('Rwanda', '371'),
 ('Kenya', 845),
 ('Albania', 2332),
 (